In [40]:
import cv2
from ultralytics import YOLO
import numpy as np

# --- CONFIGURATION ---
MODEL_PATH = r'E:\elevatetrsest\CamEDGE counting\bestv2.pt' # Use your model path
# Define the classes your model was trained on
CLASSES = ['box', 'box_broken', 'forklift', 'open_package', 'package', 'pallets', 'person'] #bestv2.pt
CLASS_TO_TRACK_EXCLUSIVELY = 'box'

# Define the coordinates for the Transfer Zone (ROI)
# Format: (x_top_left, y_top_left, width, height)
# YOU MUST ADJUST THESE VALUES BASED ON YOUR VIDEO!
TRANSFER_ZONE_X = 120
TRANSFER_ZONE_Y = 140
TRANSFER_ZONE_W = 1500
TRANSFER_ZONE_H = 450
TRANSFER_ZONE_COORDS = (TRANSFER_ZONE_X, TRANSFER_ZONE_Y, TRANSFER_ZONE_X + TRANSFER_ZONE_W, TRANSFER_ZONE_Y + TRANSFER_ZONE_H)
# --- END CONFIGURATION ---

# Load your trained model
model = YOLO(MODEL_PATH)

# For managing our own sequential box IDs
box_id_map = {}  # Maps original tracker ID of a box to our sequential display ID
next_display_box_id = 1 # Counter for our sequential display IDs

# For tracking box states relative to the ROI and counting loaded/unloaded
box_zone_tracking_info = {} # {display_id: {"was_in_zone": False}}
loaded_count = 0
unloaded_count = 0

def is_centroid_in_zone(cx, cy, zone_coords):
    """Checks if a centroid (cx, cy) is inside the defined zone_coords (x1, y1, x2, y2)."""
    x1, y1, x2, y2 = zone_coords
    return x1 < cx < x2 and y1 < cy < y2

def detect_and_custom_track_video(video_path, output_path=None, conf_threshold=0.55, tracker_config="bytetrack.yaml"):
    global box_id_map, next_display_box_id # Sequential Box IDs
    global box_zone_tracking_info, loaded_count, unloaded_count # Zone tracking and counts

    # Reset global states for each video processing call
    box_id_map.clear()
    next_display_box_id = 1
    box_zone_tracking_info.clear()
    loaded_count = 0
    unloaded_count = 0

    cap = cv2.VideoCapture(video_path)
    writer = None

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None, None

    if output_path:
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    current_frame_object_counts = {class_name: 0 for class_name in CLASSES}
    overall_objects_summary = {class_name: 0 for class_name in CLASSES}
    overall_seen_tracker_ids = set()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break
        frame_count += 1
        annotated_frame = frame.copy()

        # Draw the Transfer Zone
        cv2.rectangle(annotated_frame,
                      (TRANSFER_ZONE_COORDS[0], TRANSFER_ZONE_COORDS[1]),
                      (TRANSFER_ZONE_COORDS[2], TRANSFER_ZONE_COORDS[3]),
                      (255, 0, 255), 2) # Magenta color for zone
        cv2.putText(annotated_frame, "Transfer Zone",
                    (TRANSFER_ZONE_COORDS[0], TRANSFER_ZONE_COORDS[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)

        results = model.track(frame, conf=conf_threshold, persist=True, tracker=tracker_config, verbose=False)
        current_frame_object_counts = {class_name: 0 for class_name in CLASSES}

        if results[0].boxes is not None:
            boxes_data = results[0].boxes.xyxy.cpu().numpy()
            confs_data = results[0].boxes.conf.cpu().numpy()
            class_ids_data = results[0].boxes.cls.cpu().numpy().astype(int)
            
            original_tracker_ids_data = None
            if results[0].boxes.id is not None:
                original_tracker_ids_data = results[0].boxes.id.cpu().numpy().astype(int)

            for i in range(len(boxes_data)):
                x1_box, y1_box, x2_box, y2_box = map(int, boxes_data[i])
                conf = confs_data[i]
                class_id = class_ids_data[i]
                
                # Ensure class_id is valid for model.names
                if class_id >= len(model.names):
                    # print(f"Warning: class_id {class_id} out of bounds for model.names (len: {len(model.names)})")
                    continue # Skip this detection
                class_name = model.names[class_id]


                current_frame_object_counts[class_name] += 1
                label = f"{class_name} {conf:.2f}"
                display_id_for_this_box = None

                if original_tracker_ids_data is not None:
                    original_tracker_id = original_tracker_ids_data[i]

                    if original_tracker_id not in overall_seen_tracker_ids:
                        overall_objects_summary[class_name] = overall_objects_summary.get(class_name, 0) + 1
                        overall_seen_tracker_ids.add(original_tracker_id)

                    if class_name == CLASS_TO_TRACK_EXCLUSIVELY:
                        if original_tracker_id not in box_id_map:
                            box_id_map[original_tracker_id] = next_display_box_id
                            next_display_box_id += 1
                        display_id_for_this_box = box_id_map[original_tracker_id]
                        label = f"ID:{display_id_for_this_box} {class_name} {conf:.2f}"

                        # --- Loading/Unloading Logic for this 'box' ---
                        cx = (x1_box + x2_box) // 2
                        cy = (y1_box + y2_box) // 2
                        
                        currently_in_zone = is_centroid_in_zone(cx, cy, TRANSFER_ZONE_COORDS)

                        if display_id_for_this_box not in box_zone_tracking_info:
                            # First time seeing this box ID, initialize its zone status
                            box_zone_tracking_info[display_id_for_this_box] = {"was_in_zone": currently_in_zone}
                        else:
                            was_in_zone = box_zone_tracking_info[display_id_for_this_box]["was_in_zone"]
                            if not was_in_zone and currently_in_zone: # Box entered the zone
                                loaded_count += 1
                                print(f"Box ID:{display_id_for_this_box} LOADED (entered zone)")
                            elif was_in_zone and not currently_in_zone: # Box exited the zone
                                unloaded_count += 1
                                print(f"Box ID:{display_id_for_this_box} UNLOADED (exited zone)")
                            
                            box_zone_tracking_info[display_id_for_this_box]["was_in_zone"] = currently_in_zone
                        # --- End Loading/Unloading Logic ---

                box_color = (0, 255, 0) # Default Green
                if class_name == "box_broken": box_color = (0,0,255)
                elif class_name == "open_package": box_color = (0,165,255)
                elif class_name == "forklift": box_color = (255,0,0)
                elif class_name == "pallets": box_color = (255,255,0)
                elif class_name == "person": box_color = (128,0,128)

                cv2.rectangle(annotated_frame, (x1_box, y1_box), (x2_box, y2_box), box_color, 2)
                cv2.putText(annotated_frame, label, (x1_box, y1_box - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)
        
        # Display Counts on Frame
        y_offset = 30
        cv2.putText(annotated_frame, f"Frame: {frame_count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        y_offset += 25
        # Loaded
        text = f"Loaded: {loaded_count}"
        (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(annotated_frame, (10, y_offset - text_h), (10 + text_w + 5, y_offset + 5), (0, 0, 0), -1)
        cv2.putText(annotated_frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset += 35

        # Unloaded
        text = f"Unloaded: {unloaded_count}"
        (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(annotated_frame, (10, y_offset - text_h), (10 + text_w + 5, y_offset + 5), (0, 0, 0), -1)
        cv2.putText(annotated_frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        y_offset += 25

        
        for cls_name_txt, count_txt in current_frame_object_counts.items():
            if count_txt > 0:
                cv2.putText(annotated_frame, f"{cls_name_txt}: {count_txt}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                y_offset += 25

        if writer:
            writer.write(annotated_frame)

        cv2.imshow(f'Warehouse Tracking (Sequential IDs for "{CLASS_TO_TRACK_EXCLUSIVELY}")', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if frame_count % 100 == 0:
            unique_box_display_count = next_display_box_id -1 if next_display_box_id > 1 else 0
            print(f"Processed {frame_count} frames. Unique '{CLASS_TO_TRACK_EXCLUSIVELY}' display count: {unique_box_display_count}. Loaded: {loaded_count}, Unloaded: {unloaded_count}. Current frame counts: {current_frame_object_counts}")

    cap.release()
    if writer:
        writer.release()
    cv2.destroyAllWindows()

    total_unique_display_boxes = next_display_box_id - 1 if next_display_box_id > 1 else 0

    print(f"\n🎉 Video processing complete!")
    print(f"📝 Total frames processed: {frame_count}")
    print(f"📦 Total unique '{CLASS_TO_TRACK_EXCLUSIVELY}' objects (sequential display IDs): {total_unique_display_boxes}")
    print(f"📥 Loaded Boxes (entries into zone): {loaded_count}")
    print(f"📤 Unloaded Boxes (exits from zone): {unloaded_count}")
    
    return total_unique_display_boxes, loaded_count, unloaded_count, overall_objects_summary



In [41]:
# --- Usage Example ---
if __name__ == "__main__":
    print(f"🚀 Warehouse Video Detection (Custom Tracking & Loading/Unloading for '{CLASS_TO_TRACK_EXCLUSIVELY}') System Ready!")
    print(f"❗ IMPORTANT: Adjust TRANSFER_ZONE_COORDS in the script for your video.")

    video_input_path = r"E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4"  # REPLACE THIS
    video_output_path = r'E:\elevatetrsest\CamEDGE counting\tracking_output\output3.mp4' # REPLACE THIS (OPTIONAL)

    if video_input_path == r'path_to_your_warehouse_video.mp4':
        print("\n⚠️ PLEASE UPDATE 'video_input_path' in the script with your actual video file path.")
    else:
        try:
            print(f"\nProcessing video: {video_input_path}")
            # unique_boxes, loaded, unloaded, _ = detect_and_custom_track_video(video_input_path, conf_threshold=0.4)
            # To save output:
            unique_boxes, loaded, unloaded, _ = detect_and_custom_track_video(video_input_path, video_output_path, conf_threshold=0.55)
            
            print(f"\nFinal count of unique '{CLASS_TO_TRACK_EXCLUSIVELY}' objects: {unique_boxes}")
            print(f"Final count of Loaded events: {loaded}")
            print(f"Final count of Unloaded events: {unloaded}")

        except FileNotFoundError:
            print(f"ERROR: Video file not found at '{video_input_path}'. Please check the path.")
        except Exception as e:
            print(f"An error occurred: {e}")
            import traceback
            traceback.print_exc()
            print("Ensure your video path is correct and OpenCV and Ultralytics are installed properly.")

🚀 Warehouse Video Detection (Custom Tracking & Loading/Unloading for 'box') System Ready!
❗ IMPORTANT: Adjust TRANSFER_ZONE_COORDS in the script for your video.

Processing video: E:\elevatetrsest\CamEDGE counting\test\test3_warehouse.mp4
Box ID:2 LOADED (entered zone)
Box ID:4 LOADED (entered zone)
Processed 100 frames. Unique 'box' display count: 4. Loaded: 2, Unloaded: 0. Current frame counts: {'box': 2, 'box_broken': 0, 'forklift': 0, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 0}
Box ID:4 UNLOADED (exited zone)
Box ID:4 LOADED (entered zone)
Box ID:5 LOADED (entered zone)
Processed 200 frames. Unique 'box' display count: 5. Loaded: 4, Unloaded: 1. Current frame counts: {'box': 1, 'box_broken': 0, 'forklift': 0, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 0}
Processed 300 frames. Unique 'box' display count: 5. Loaded: 4, Unloaded: 1. Current frame counts: {'box': 0, 'box_broken': 0, 'forklift': 0, 'open_package': 0, 'package': 0, 'pallets': 0, 'person': 1}